In [115]:
import scipy.io
import numpy as np
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import sys
import timeit
from sklearn.metrics import confusion_matrix
import math
from skimage.transform import rotate
import scipy.fftpack as fft

from skimage.metrics import peak_signal_noise_ratio as psnr
from skimage.metrics import structural_similarity as ssim

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

In [ ]:
input = scipy.io.loadmat("/content/drive/MyDrive/Data/HW/ctscan_hw1.mat")
ct_scans = input['ctscan']
ct_scans = np.array(ct_scans)
ct_scans = np.swapaxes(ct_scans,0,2)
ct_scans = np.swapaxes(ct_scans,1,2)
print(ct_scans.shape)

In [ ]:
input = scipy.io.loadmat("/content/drive/MyDrive/Data/HW/infmsk_hw1.mat")
mask = input['infmsk']
mask = np.array(mask)
mask = np.swapaxes(mask,0,2)
mask = np.swapaxes(mask,1,2)
print(mask.shape)

In [ ]:
mask=mask[1010:1020,:,:]
ct_scans=ct_scans[1010:1020,:,:]
size = ct_scans.shape[0]
print(size)

In [154]:
from sklearn.cluster import KMeans 
from numpy.ma.core import argsort
def predicted_mask(ct_scans):
  pred_masks = []
  a=ct_scans.shape[1]
  b=ct_scans.shape[2]
  for i in range(ct_scans.shape[0]):
    temp = ct_scans[i]/255
    kmeans = KMeans(n_clusters=3, random_state=0)
    clusters = kmeans.fit_predict(temp.reshape(a*b,1))
    temp_pred_mask = clusters.reshape((512,512))
    copy=temp_pred_mask
    ct=[]
    ct.append(np.count_nonzero(temp_pred_mask[:,:] == 0))
    ct.append(np.count_nonzero(temp_pred_mask[:,:] == 1))
    ct.append(np.count_nonzero(temp_pred_mask[:,:] == 2))

    max_val=max(ct)
    b1=ct.index(max_val)
    mask1=(temp_pred_mask==b1)
    copy[mask1]=0

    min_val=min(ct)
    b2=ct.index(min_val)
    mask2=(temp_pred_mask==b2)
    copy[mask2]=1

    b3=3-b1-b2
    mask3=(temp_pred_mask==b3)
    copy[mask3]=2

    pred_masks.append(copy)
  return pred_masks

In [155]:
#Convert image into a sinogram
def radon_transform(img, rots = 4): # rots is 4X or 8X
    from skimage.transform import rotate
    #Turns image into sinogram
    projections = []
    dTheta = -rots
    steps = int(180/rots)

    for i in range(steps):
        projections.append(rotate(img,i*dTheta).sum(axis=0))
    return np.vstack(projections)

# def fft_translate(projs):
#     #Build 1-d FFTs of an array of projections, each projection 1 row of the array.
#     return fft.rfft(projs, axis=1)

# def ramp_filter(ffts):
#     #Ramp filter a 2-d array of 1-d FFTs (1-d FFTs along the rows).
#     ramp = np.floor(np.arange(0.5, ffts.shape[1]//2 + 0.1, 0.5))
#     return ffts * ramp

# def inverse_fft_translate(operator):
#     return fft.irfft(operator, axis=1)

def back_project(operator): 
  operator = fft.rfft(operator, axis=1)   
  operator = operator*np.floor(np.arange(0.5, operator.shape[1]//2 + 0.1, 0.5))   #ramp_filter(operator)
  operator = fft.irfft(operator, axis=1)   #inverse_fft_translate(operator)
  laminogram = np.zeros((operator.shape[1],operator.shape[1]))
  dTheta = 180.0 / operator.shape[0]
  for i in range(operator.shape[0]):
    temp = np.tile(operator[i],(operator.shape[1],1))
    temp = rotate(temp, dTheta*i)
    laminogram += temp
  return laminogram

In [156]:
reconstructed_ct_scans_4x = []
from skimage.transform import radon, rescale, iradon
for i in range(size):
  # print(i)
  # t=radon_transform(ct_scans[i], rots = 4)
  t=radon(ct_scans[i],circle=False,preserve_range=True)
  t = np.array([t[:, i] for i in range(0, 180, 4)])
  t2 = iradon(t.T, circle=False, preserve_range=True)
  # t2=back_project(t)
  reconstructed_ct_scans_4x.append(t2)

reconstructed_ct_scans_8x = []
for i in range(size):
  # print(i)
  # reconstructed_ct_scans_8x.append(back_project(radon_transform(ct_scans[i], rots = 8)))
  t=radon(ct_scans[i],circle=False,preserve_range=True)
  t = np.array([t[:, i] for i in range(0, 180, 8)])
  t2 = iradon(t.T, circle=False, preserve_range=True)
  reconstructed_ct_scans_8x.append(t2)

In [ ]:
avg_psnr_4x = 0
avg_ssim_4x = 0
for i in range(size):
  avg_psnr_4x += psnr(ct_scans[i],reconstructed_ct_scans_4x[i])
  avg_ssim_4x += ssim(ct_scans[i],reconstructed_ct_scans_4x[i])
    
avg_psnr_4x = avg_psnr_4x/size
avg_ssim_4x = avg_ssim_4x/size

print("PSNR 4x Reconstruction: ", avg_psnr_4x)
print("SSIM 4x Reconstruction: ", avg_ssim_4x)

In [158]:
from sklearn.metrics import confusion_matrix
import warnings
warnings.filterwarnings('ignore')
def give_metrics(y, y_pred):
  y_pred = np.array(y_pred)
  N = y_pred.shape[0]
  total_infection_sensitivity = 0
  total_infection_specificity = 0
  total_infection_accuracy = 0
  total_infection_dice_score = 0
  
  total_normal_sensitivity = 0
  total_normal_specificity = 0
  total_normal_accuracy = 0
  total_normal_dice_score = 0

  count_infection_sensitivity = 0               

  for i in range(y_pred.shape[0]):
    infection_sensitivity = 0
    #Getting confusion matrix
    confusion_metric = (confusion_matrix(y[i].flatten(), y_pred[i].flatten(),labels=[0,1,2])).T

    #Getting TP/FP/FN/TN
    TP_infection = confusion_metric[1][1]
    TP_normal = confusion_metric[2][2]

    TN_infection = confusion_metric[0][0] + confusion_metric[2][0] + confusion_metric[0][2] + confusion_metric[2][2]
    TN_normal = confusion_metric[0][0] + confusion_metric[0][1] + confusion_metric[1][0] + confusion_metric[1][1]
    
    FP_infection = confusion_metric[1][0] + confusion_metric[1][2]
    FP_normal = confusion_metric[2][0] + confusion_metric[2][1] 

    FN_infection = confusion_metric[0][1] + confusion_metric[2][1]
    FN_normal = confusion_metric[0][2] + confusion_metric[1][2]
        
    #calculating metrics    
    infection_specificity = (TN_infection)/(TN_infection+FP_infection)
    infection_accuracy = (TP_infection+TN_infection)/(TP_infection+TN_infection+FP_infection+FN_infection)
    infection_dice_score = (2*TP_infection)/(2*TP_infection + FP_infection + FN_infection)

    normal_sensitivity = (TP_normal)/(TP_normal+FN_normal)
    normal_specificity = (TN_normal)/(TN_normal+FP_normal)
    normal_accuracy = (TP_normal+TN_normal)/(TP_normal+TN_normal+FP_normal+FN_normal)
    normal_dice_score = (2*TP_normal)/(2*TP_normal + FP_normal + FN_normal)
    
    if((TP_infection+FN_infection)!=0):
      count_infection_sensitivity += 1
      infection_sensitivity = (TP_infection)/(TP_infection+FN_infection)
    
    # normal_sensitivity = 0
    # if((normal_TP+infectio_FN)!=0):
    #   count_infection_sensitivity += 1
    #   infection_sensitivity = (infection_TP)/(infection_TP+infection_FN)

    total_infection_sensitivity += infection_sensitivity
    total_normal_sensitivity += normal_sensitivity
    total_infection_specificity += infection_specificity
    total_normal_specificity += normal_specificity
    total_infection_accuracy += infection_accuracy
    total_normal_accuracy += normal_accuracy
    total_infection_dice_score += infection_dice_score
    total_normal_dice_score += normal_dice_score
  #finding avg and printing 
  print(f"Averaged dice score = {total_normal_dice_score/N}, sensitivity = {total_normal_sensitivity/N}, \n specificity = {total_normal_specificity/N}, and accuracy = {total_normal_accuracy/N} for the normal")
  print(f"\nAveraged dice score = {total_infection_dice_score/N}")
  print(f", sensitivity = {total_infection_sensitivity/(count_infection_sensitivity+0.0001)}")
  print(f"\n specificity = {total_infection_specificity/N},")
  print(f"and accuracy = {total_infection_accuracy/N} for the infection")

In [ ]:
avg_psnr_8x = 0
avg_ssim_8x = 0
for i in range(size):
  avg_psnr_8x += psnr(ct_scans[i],reconstructed_ct_scans_8x[i])
  avg_ssim_8x += ssim(ct_scans[i],reconstructed_ct_scans_8x[i])
    
avg_psnr_8x = avg_psnr_8x/size
avg_ssim_8x = avg_ssim_8x/size

print("PSNR 8x Reconstruction: ", avg_psnr_8x)
print("SSIM 8x Reconstruction: ", avg_ssim_8x)

In [ ]:
# print(reconstruct_4x.reconstructed_ct_scans)
pred_masks_4x = predicted_mask(np.array(reconstructed_ct_scans_4x))
# print(pred_masks_4x)
print("Evaluation Metrics for 4x Reconstruction")
pred_masks_8x = predicted_mask(np.array(reconstructed_ct_scans_8x))
print("Evaluation Metrics for 8x Reconstruction")

In [ ]:

give_metrics(mask, np.array(pred_masks_4x))
give_metrics(mask, np.array(pred_masks_8x))


In [ ]:
plt.rcParams["figure.figsize"] = (12,12)
i = 5
f, axarr = plt.subplots(1,3)
axarr[0].set_title("4x Reconstruction")
axarr[0].imshow(reconstructed_ct_scans_4x[i], cmap='gray')
axarr[1].set_title("8x Reconstruction")
axarr[1].imshow(reconstructed_ct_scans_8x[i], cmap='gray')
axarr[2].set_title("CT Scans")
axarr[2].imshow(ct_scans[i], cmap='gray')
f.tight_layout()
plt.show()